In [0]:
%pip install apache-airflow
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime, timedelta
# from airflow.providers.standard.operators.python.PythonOperator import PythonOperator



In [0]:
from helper_logger import LoggerManager
from helper_yaml import *
from helper_misc import *
from helper_ObjectsTracker import *

In [0]:
# logger_for_initializer = LoggerManager(
#     logger_name='logger_for_initializer'
#     ,log_file='/Workspace/Users/akshay.chidrawar@ltimindtree.com/bookstore/log/test.log'
#     ,log_mode='w')

# def task_log_details(**context,logger=logger_for_initializer):
#     logger.info(f"Task: {context['task_instance'].task_id}")

def task_wrapper(task_func):
    def wrapper(*args,**kwargs):
        context = kwargs.get('context',{})
        ti = context.get('ti')
        try:
            return task_func(*args,**kwargs)
        except Exception as e:
            error_details = {
                'error_type': type(e).__name__,
                'error_message': str(e)
            }
            if ti:
                ti.xcom_push(key='error_details',value=error_details)
            raise
    return wrapper

def task_get_repository(repository_file,**context):
    repository = read_yaml(repository_file)
    home = get_path(repository['project']['workspacehome'],repository['project']['reponame'])
    dict_repo = {
        "path_ddl":get_path(home,repository['path']['ddl'])
        ,"path_schema": get_path(home,repository['path']['schema'])
        ,"path_log": get_path(home,repository['path']['log'])
    }
    context['ti'].xcom_push(key='dict_repo',value=dict_repo)

def task_get_metadata(metadata_file,**context):
    metadata = read_yaml(metadata_file)
    dict_meta = {
        "host":metadata['project']['host']
        ,"token":metadata['project']['token']
        ,"buckets":metadata['buckets']
        ,"dwh_structure":metadata['dwh_structure']
        ,"table_schemas": metadata['table_schemas']
        ,"objects_tracker": metadata['objects_tracker']
    }
    context['ti'].xcom_push(key='dict_meta',value=dict_meta)

def task_create_workspace(**context):
    workspace = create_Workspace(
            host=dict_meta['host'],
            token=dict_meta['token']
        )
    context['ti'].xcom_push(key='workspace',value=workspace)

def task_create_objects_tracker(**context):
    manage_objects_tracker(DDLType='CREATE')

default_args = {
    'owner': 'airflow',
    'start_date': datetime(2024, 6, 1),
    'retries': 1,
    'retry_delay': timedelta(minutes=5),
    # "on_failure_callback": failure_callback,
    # "on_success_callback": success_callback,
    # "on_retry_callback": retry_callback,
    "email_on_failure": False,
    "email_on_retry": False
}

with DAG(
    'initializer_dag',
    default_args=default_args,
    schedule=None,
    catchup=False
) as dag:

    get_repository = PythonOperator(
        task_id='get_repository',
        python_callable=task_wrapper(task_get_repository),
        op_kwargs={'repository_file': '/Workspace/Users/akshay.chidrawar@ltimindtree.com/bookstore/config/repository.yaml'},
    )
    get_metadata = PythonOperator(
        task_id='get_metadata',
        python_callable=task_wrapper(task_get_metadata),
        op_kwargs={'repository_file': '/Workspace/Users/akshay.chidrawar@ltimindtree.com/bookstore/config/metadata.yaml'},
    )
    create_workspace = PythonOperator(
        task_id='create_workspace',
        python_callable=task_wrapper(task_create_workspace),
    )
    create_objects_tracker = PythonOperator(
        task_id='create_objects_tracker',
        python_callable=task_wrapper(task_create_objects_tracker),
    )

    [get_repository,get_metadata,create_objects_tracker]
    get_metadata >> create_workspace

In [0]:
def get_xcom_values(**context):
    value = context['ti'].xcom_pull(
        task_id='task_name',
        key='key_name'
    )

print(get_xcom_values(**{'ti':get_metadata})

In [0]:
def Initialize(self):
    try:
        self.dict_repo = self.get_Repository(logger = self.logger_for_initializer)
        self.dict_meta = self.get_Metadata(logger = self.logger_for_initializer)
        self.workspace = create_Workspace(host=self.dict_meta['host'],token=self.dict_meta['token'],logger=self.logger_for_initializer)
        self.ObjectsTracker.manage_objects_tracker(DDLType='CREATE',logger=self.logger_for_initializer)
        self.logger_for_initializer.info(f"Success: Initializer succecceded.")
        print(f"\nInitializer succecceded.")
    except Exception as e:
        self.logger_for_initializer.error(f"Failure: Error occurred for initializer: \n{e}")
        print(f"\nInitializer failed.")
    print(f"For more details, refer below log file.\n'{get_log_file(self.logger_for_initializer)}'")




